
# [ParslFest 2025](https://parsl-project.org/parslfest/parslfest2025.html)

# [Accelerating QMCpy Notebook Tests with Parsl](https://www.figma.com/slides/k7EUosssNluMihkYTLuh1F/Parsl-Testbook-Speedup?node-id=1-37&t=WnKcu2QYO8JXvtpP-0)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QMCSoftware/QMCSoftware/blob/develop/demos/parsl_fest_2025.ipynb)

Joshua Herman and Sou-Cheng Choi, QMCPy Developers

Aug 28 -- 29, 2025

Updated: Sep 9, 2025

**Requirements**:

* QMCPy: `pip install qmcpy==2.1`
* LaTeX: `sudo apt update && sudo apt install -y texlive-full`
* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

This notebook can be run interactively or in command line mode. To run in command line mode, use:
```bash
    conda activate qmcpy && cd demos && jupyter execute --kernel_name qmcpy parsl_fest_2025.ipynb
```

Our presentation slides for ParslFest are available at [Figma](https://www.figma.com/slides/k7EUosssNluMihkYTLuh1F/Parsl-Testbook-Speedup?node-id=174-95&t=t3jENVMltXWwdLdb-0).

In [1]:
try:
    import qmcpy as qp
except ModuleNotFoundError:
    !pip install -q qmcpy

try:
    import testbook as tb
except ModuleNotFoundError:
    !pip install -q testbook

In [2]:
import sys
import os
import time
import inspect
import parsl as pl
from parsl.configs.htex_local import config

## 1. Sequential Execution

In [3]:
start_time = time.time()
!(cd .. && make booktests_no_docker)
sequential_time = time.time() - start_time

find demos -name '*.ipynb' | while read nb; do \
		base=$(basename "$nb" .ipynb); \
		test_base=$(echo "$base" | sed 's/[-.]/_/g'); \
		if [ "$base" != "parsl_fest_2025" ] && ! ls test/booktests/tb_"$test_base".py &>/dev/null; then \
			echo "Missing test for: $nb -> Expected: test/booktests/tb_$test_base.py"; \
		fi; \
	done
Total notebooks:        33
Total test files:       32

Generating missing booktest files...
cd test/booktests/ && python generate_test.py --check-missing
No missing test files found.
chmod +x scripts/find_local_only_folders.sh
for f in .coverage .pdm-build/.gitignore .pdm-build/qmcpy.pth .pytest_cache/v/cache/nodeids .pytest_cache/v/cache/stepwise __pycache__/__init__.cpython-39.pyc __pycache__/pdm_build.cpython-39.pyc build/lib.macosx-11.0-arm64-cpython-39/qmcpy/discrete_distribution/_c_lib/_c_lib.cpython-39-darwin.so build/temp.macosx-11.0-arm64-cpython-39/qmcpy/discrete_distribution/_c_lib/halton_qrng.o build/temp.macosx-11.0-arm64-cpython-39/qmcpy/discrete_dis

## 2. Parsl

1. Install and Configure Parsl
2. Run the tests in Parallel with Parsl

### 2.1 Install and Configure Parsl

In [4]:
try:
    import parsl as pl
except ModuleNotFoundError:
    !pip install -q parsl
    
config.max_workers = 8  
if not pl.config:
    pl.load(config)
    print("Parsl loaded with memory limits")
else:
    print("Parsl already configured")

Parsl already configured


### 2.2 Create a Parsl Test Runner

In [5]:
# Ensure the path to the booktests directory is included
sys.path.append(os.path.join(os.getcwd(), '..', 'test', 'booktests'))

In [6]:
import parsl_test_runner
import inspect

# See only functions
print("Functions:")
functions = inspect.getmembers(parsl_test_runner, inspect.isfunction)
for name, func in functions:
    print(f"- {name}")
print("\n" + "="*50)

# Get help on specific function
print("Help for execute_parallel_tests:")
help(parsl_test_runner.execute_parallel_tests)

Functions:
- bash_app
- execute_parallel_tests
- generate_summary_report
- main

Help for execute_parallel_tests:
Help on function execute_parallel_tests in module parsl_test_runner:

execute_parallel_tests()
    Execute all testbook tests in parallel using Parsl



### 2.3 Run the Notebooks in Parallel with Parsl

In [7]:
start_time2 = time.time()
!(cd .. && make booktests_parallel_no_docker)
parallel_time = time.time() - start_time2

find demos -name '*.ipynb' | while read nb; do \
		base=$(basename "$nb" .ipynb); \
		test_base=$(echo "$base" | sed 's/[-.]/_/g'); \
		if [ "$base" != "parsl_fest_2025" ] && ! ls test/booktests/tb_"$test_base".py &>/dev/null; then \
			echo "Missing test for: $nb -> Expected: test/booktests/tb_$test_base.py"; \
		fi; \
	done
Total notebooks:        33
Total test files:       32

Generating missing booktest files...
cd test/booktests/ && python generate_test.py --check-missing
No missing test files found.
chmod +x scripts/find_local_only_folders.sh
for f in .pdm-build/.gitignore .pdm-build/qmcpy.pth __pycache__/pdm_build.cpython-39.pyc build/lib.macosx-11.0-arm64-cpython-39/qmcpy/discrete_distribution/_c_lib/_c_lib.cpython-39-darwin.so build/temp.macosx-11.0-arm64-cpython-39/qmcpy/discrete_distribution/_c_lib/halton_qrng.o build/temp.macosx-11.0-arm64-cpython-39/qmcpy/discrete_distribution/_c_lib/util.o qmcpy/__pycache__/__init__.cpython-39.pyc qmcpy/discrete_distribution/__pycache__/_

In [8]:
speedup = sequential_time / parallel_time
print(f"Sequential time:  {sequential_time:.2f} seconds")
print(f"Parallel time:    {parallel_time:.2f} seconds")
print(f"Speedup by Parsl: {speedup:.2f}")

Sequential time:  1527.49 seconds
Parallel time:    1107.81 seconds
Speedup by Parsl: 1.38


In [9]:
import platform

if platform.system().lower() == 'linux':
    !uname -a
    !nproc --all
    !awk '/MemTotal/ {printf "%.2f GB\n", $2/1024/1024}' /proc/meminfo